In [10]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [11]:
# read picture as GRAYSCALE
img=cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)

In [12]:
# binary image (white->1 black->0)
def binarize(image):
    answer=np.zeros(image.shape, dtype=np.uint8)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i][j]>127:
                answer[i][j] = 1
            else:
                answer[i][j] = 0
    return answer

In [13]:
# top-left downsample image(8*8)
def Top_left_DownSample(binary_image):
    new_shape_0 = int(binary_image.shape[0]/8)
    new_shape_1 = int(binary_image.shape[1]/8)
    return_matrix = np.zeros((new_shape_0,new_shape_1))
    
    for i in range(0,binary_image.shape[0],8):
        for j in range(0,binary_image.shape[1],8):
            if binary_image[i][j] == 1:
                return_matrix[int(i/8)][int(j/8)] = 1
                
    return return_matrix
                
            

In [14]:
# decide the corner type
def Yokoi_corner_type(b,c,d,e):
    if b==c and (d!=b or e!=b):
        return 'q'
    elif b==c and d==b and e==b:
        return 'r'
    elif b!=c:
        return 's'
    else:
        print("error! not a corner type!")

In [15]:
# decide the Yokoi number
def Yokoi_number(sub_image):
    dictionary = {'q': 0, 'r': 0, 's': 0}
    
    # top_right
    dictionary[Yokoi_corner_type(sub_image[1][1],sub_image[1][2],sub_image[0][2],sub_image[0][1])]+=1
    # top_left
    dictionary[Yokoi_corner_type(sub_image[1][1],sub_image[0][1],sub_image[0][0],sub_image[1][0])]+=1
    # buttom_left
    dictionary[Yokoi_corner_type(sub_image[1][1],sub_image[1][0],sub_image[2][0],sub_image[2][1])]+=1
    # buttom_right
    dictionary[Yokoi_corner_type(sub_image[1][1],sub_image[2][1],sub_image[2][2],sub_image[1][2])]+=1
    
    if dictionary['s'] == 4:
        return 0
    elif dictionary['q'] == 1:
        return 1
    elif dictionary['q'] == 2:
        return 2
    elif dictionary['q'] == 3:
        return 3
    elif dictionary['q'] == 4:
        return 4
    elif dictionary['r'] == 4:
        return 5
    else:
        print("error! not a yokoi number!")
        

In [16]:
def Yokoi(image):
    binary_image = binarize(image)
    down_sample_image = Top_left_DownSample(binary_image)
    
    result_shape_0 = down_sample_image.shape[0]
    result_shape_1 = down_sample_image.shape[1]
    
    zero_padding_down_sample_image = np.zeros((result_shape_0 + 2,result_shape_1 + 2),dtype=int)
    zero_padding_down_sample_image[1:result_shape_0 + 1,1:result_shape_1 + 1] = down_sample_image[0:result_shape_0][0:result_shape_0]
    
    return_matrix = np.zeros((result_shape_0,result_shape_1),dtype=int)
    return_matrix.fill(-1)
    
    for i in range(1,result_shape_0 + 1):
        for j in range(1,result_shape_1 + 1):
            if zero_padding_down_sample_image[i][j] == 1:
                return_matrix[i-1][j-1] =  Yokoi_number(zero_padding_down_sample_image[i-1:i+2,j-1:j+2])
    
    return return_matrix

In [17]:
result = Yokoi(img)
result_string = ""

for i in range(result.shape[0]):
    for j in range(result.shape[1]):
        if result[i][j]>0:
            result_string = result_string + str(result[i][j])
        else:
            result_string = result_string + " "
    result_string = result_string + "\n"
    
print(result_string)

11111111        12111111111122322221     111111111111           
15555551         115555555511 2 11  11   1155555555511          
15555551        1 2115555112  21112221    155555555551      21  
15555551        1 2 155112 22221511       1555555555511     1   
15555551         22 2112 22    121        15555555555511        
15555551         1  2  21 2     1   1     15555555555551        
15555551           12 1  121111    1321   155555555555511       
15111551           1322 1155551111        155555555555551       
111 1551            1  121555555511       155555555555511       
11  1551                 21155555511      15511155555511        
21  1551                 2 15555555111    1551 11555511         
1   1551                 2 155555555511   1551  115551         1
    1551               1121155555555551   1551   15511        12
    1551               15555555555555511  1551   1111        111
    1551        1     2221155555555555511 1151    11        1151
    1551        2    22 1